In [0]:
from __future__ import print_function

import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten,Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import RMSprop
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

batch_size = 128
num_classes = 10
epochs = 50

img_rows, img_cols = 28, 28
nb_train_samples = 60000
nb_validation_samples = 10000

################## Def dataset ###########################

#the data, split between train and test sets

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

#x_train = x_train.reshape(60000, 56)
#x_test = x_test.reshape(10000, 56)

if K.image_data_format() == 'channels first':
  x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
  x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
  input_shape = (1, img_rows, img_cols)
else:
  x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
  x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
  input_shape = (img_rows, img_cols, 1)
  
  
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#to ensure a good dynamic, help the learnig convergence
x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#convert class vectors to binary class matrices

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

################## Data augmentation ###################

train_datagen = ImageDataGenerator(horizontal_flip=True)

train_generator = train_datagen.flow(
  x_train,
  y_train,
  batch_size=batch_size)

################## Def model ###########################

model = Sequential()
"""
model.add(Conv2D(16, kernel_size=(3,3),
                activation='relu',
                input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, kernel_size=(3,3),
                activation='relu',
                input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(16, kernel_size=(1,1),
                activation='relu',
                input_shape=input_shape))
model.add(Conv2D(128, kernel_size=(3,3),
                activation='relu',
                input_shape=input_shape))
model.add(Conv2D(16, kernel_size=(1,1),
                activation='relu',
                input_shape=input_shape))
model.add(Conv2D(128, kernel_size=(3,3),
                activation='relu',
                input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
"""
model.add(Conv2D(32, kernel_size=(1,1),
                activation='relu',
                input_shape=input_shape))
model.add(Conv2D(256, kernel_size=(3,3),
                activation='relu',
                input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(1,1),
                activation='relu',
                input_shape=input_shape))
model.add(Conv2D(256, kernel_size=(3,3),
                activation='relu',
                input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(1,1),
                activation='relu',
                input_shape=input_shape))
model.add(Conv2D(512, kernel_size=(3,3),
                activation='relu',
                input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(1,1),
                activation='relu',
                input_shape=input_shape))
model.add(Conv2D(512, kernel_size=(3,3),
                activation='relu',
                input_shape=input_shape))
model.add(Conv2D(128, kernel_size=(1,1),
                activation='relu',
                input_shape=input_shape))
model.add(Conv2D(1000, kernel_size=(1,1),
                activation='relu',
                input_shape=input_shape))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_classes, activation='softmax'))

#model.add(BatchNormalization())
  
model.summary()

model.compile(loss='categorical_crossentropy', optimizer = RMSprop(), metrics=['accuracy'])


################## Learning ###########################

#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)


# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
"""
earlystopping = EarlyStopping(monitor='val_acc',
                              min_delta=0,
                              patience=5,
                              verbose=1, mode='auto')
callbacks_list = [checkpoint, earlystopping]
"""
callbacks_list = [checkpoint]

#fit permet de lancer l'apprentissage, avec les donnees, les labels, la taille du batch, le nb d'epoque, la verbosite, et les donnees de validation d'epoque a epoque
"""
history = model.fit(x_train, y_train, 
                     batch_size=batch_size, 
                     epochs=epochs, 
                     verbose=1, 
                     validation_data=(x_test, y_test),
                     callbacks=callbacks_list)
"""

history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test),
    callbacks=callbacks_list)

#evaluate toward test dataset
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

model.save_weights('fashion_try_0110.h5')
model.save('Test_FASHION_MNIST_V1.h5')

60000 train samples
10000 test samples
Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_74 (Conv2D)           (None, 28, 28, 32)        64        
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 26, 26, 256)       73984     
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 26, 26, 32)        8224      
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 24, 24, 256)       73984     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 12, 12, 64)        16448     
_________________________________________________________________
conv2d_79 (Con

model from : https://www.dlology.com/blog/one-simple-trick-to-train-keras-model-faster-with-batch-normalization/

Epoch 00050: val_acc did not improve from 0.90950

Test loss:  0.7327184855340101

Test accuracy:  0.902





#En rajoutant Dense 128

Epoch 00050: val_acc did not improve from 0.91210# 

Test loss:  0.7428650218252558

Test accuracy:  0.9072



#En rajoutant Dense 256

Total params: 246,008

Epoch 00050: val_acc did not improvefrom 0.91430

Test loss:  0.6275242215685546

Test accuracy:  0.9134




#Dernier résultat :

Epoch 00050: val_acc did not improve from 0.91700

Test loss:  0.7110278423965914

Test accuracy:  0.9069

#Data Augmentation
En ajoutant de la data augmentation via un horizontal_flip : 

Total params : 525,048

Test loss:  0.3304

Test accuracy:  0.9150